<a href="https://colab.research.google.com/github/Qv-code-Qv/kayak_data/blob/main/booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy

In [ ]:
!python booking.py

In [1]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

In [44]:
df2 = pd.read_csv('gps_data_1.csv')
df2.head()
df1 = pd.read_csv('gps_data.csv')
df1.head()

fig = make_subplots(rows=1, cols=2)

fig1 = px.density_mapbox(df1, opacity=1, title="Météo de la semaine :", lat="Latitude", lon="Longitude",
                        zoom=4, mapbox_style="open-street-map")


fig2 = px.scatter_mapbox(df2, opacity=1, title="Nos meilleurs destinations pour le week-end :", color="Ville", text="Hotel",  lat="Latitude", lon="Longitude",
                        zoom=4, mapbox_style="open-street-map")

#fig = fig1.add_trace(fig2.data[0])
fig1.update_layout(height=450, width=716, title_text="Météo de la semaine :")

fig2.update_layout(height=450, width=716, title_text="Nos meilleurs destinations pour le week-end :")
#fig.show()
fig1.show()
fig2.show()
"""
fig.add_trace(
    fig1.data[0],
    row=1,
    col=1
)
fig.add_trace(
    fig2.data[0],
    row=1,
    col=2
)
fig
"""

'\nfig.add_trace(\n    fig1.data[0],\n    row=1,\n    col=1\n)\nfig.add_trace(\n    fig2.data[0],\n    row=1,\n    col=2\n)\nfig\n'

In [ ]:
import os 
import logging
import boto3.session
import my_key as key
import pandas as pd
import logging
import boto3
import sqlite3

AWS_ACCESS_KEY_ID = key.aws_access_key_id
AWS_SECRET_ACCESS_KEY =key.aws_secret_access_key
REGION_NAME = "eu-west-3"
BUCKET_NAME ="jehda-2024-ps-kayak.project"

booking_json = open('fichier_final/booking.json', 'rb')

#sauvegarde du résultat dans data lake (aws-s3)
logging.getLogger('botocore').setLevel(logging.INFO)
logging.getLogger('boto3').setLevel(logging.INFO)
session =   boto3.Session(aws_access_key_id = AWS_ACCESS_KEY_ID,
                        aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                        region_name= REGION_NAME)                  
s3_kayak = session.resource("s3")

""" CREATE BUCKET AND UPLOAD FILE """
bucket_kayak = s3_kayak.create_bucket(Bucket=BUCKET_NAME,
                          CreateBucketConfiguration={'LocationConstraint':REGION_NAME})
print("Création du bucket Jedha dans AWS :", bucket_kayak, "\n")
bucket_kayak.upload_file("fichier_final/booking.json","files/booking.json")

""" DOWNLOAD JSON FILE FROM S3 AND READ AS  DATAFRAME """
s3_obj = s3_kayak.Bucket(BUCKET_NAME).Object("files/booking.json").get()
booking_df = pd.read_json(s3_obj['Body'])
print(booking_df.head())

In [ ]:
#CREATE Database booking
conn = sqlite3.connect(':memory:')
c= conn.cursor()

#CREATE TABLE booking
booking_df.to_sql('booking', conn, if_exists='replace', index = False)
c.execute('SELECT ville, name, score FROM booking LIMIT 5')
for row in c.fetchall():
    print (row)
conn.close()

In [ ]:
""" SUPPRESSION JSON FILE in AWS  """
response = bucket_kayak.delete_objects(
    Delete={
        'Objects': [
            {
                'Key': "files/booking.json"
            }
        ]
    }
)
print ("\nconfirmation suppression booking.json file dans AWS", response, "\n")

response = bucket_kayak.delete()
print ("confirmation suppression bucket jedha-bucketdeletion dans AWS", response, "\n")

In [72]:
import pandas as pd
import json
from operator import itemgetter
booking_in = pd.read_json("booking1.json")
tmp =booking_in.sort_values("ville")
print(tmp)
tmp.to_json("booking2.json", orient="records")
booking_sorted = pd.read_json("booking2.json")
booking_sorted


                                               ville  \
257  Aigues-Mortes%2C+Languedoc-Roussillon%2C+France   
254  Aigues-Mortes%2C+Languedoc-Roussillon%2C+France   
255  Aigues-Mortes%2C+Languedoc-Roussillon%2C+France   
286  Aigues-Mortes%2C+Languedoc-Roussillon%2C+France   
256  Aigues-Mortes%2C+Languedoc-Roussillon%2C+France   
..                                               ...   
394                                             Uzes   
395                                             Uzes   
396                                             Uzes   
387                                             Uzes   
377                                             Uzes   

                                                  name  \
257                                  le dit vin secret   
254                                  La Maison de Lyna   
255                               Ô 36 Rempart Sud B&B   
286                                  Hôtel Le Médiéval   
256                               Mai

,ville,name,website,score,desc,lat,lon
0,Aigues-Mortes%2C+Languedoc-Roussillon%2C+France,le dit vin secret,https://www.booking.com/hotel/fr/le-dit-vin-se...,3 sur 5,Hébergement géré par un particulier,43.566769,4.189113
1,Aigues-Mortes%2C+Languedoc-Roussillon%2C+France,La Maison de Lyna,https://www.booking.com/hotel/fr/la-maison-de-...,3 sur 5,"Situé à Aigues-Mortes, à 24 km de la salle omn...",43.566018,4.192353
2,Aigues-Mortes%2C+Languedoc-Roussillon%2C+France,Ô 36 Rempart Sud B&B,https://www.booking.com/hotel/fr/o-36-rempart-...,3 sur 5,Hébergement géré par un particulier,43.565764,4.188757
3,Aigues-Mortes%2C+Languedoc-Roussillon%2C+France,Hôtel Le Médiéval,https://www.booking.com/hotel/fr/le-medieval.f...,2 sur 5,L'Hôtel Le Médiéval est situé sur les rives du...,43.571866,4.193662
4,Aigues-Mortes%2C+Languedoc-Roussillon%2C+France,Maison des Croisades,https://www.booking.com/hotel/fr/des-croisades...,3 sur 5,"Située à Aigues-Mortes, la Maison des Croisade...",43.568997,4.188298
...,...,...,...,...,...,...,...
695,Uzes,Hôtel Le Saint-Geniès,https://www.booking.com/hotel/fr/le-saint-geni...,3 sur 5,Cet hôtel à la gestion familiale est situé au ...,44.023053,4.420472
696,Uzes,Le trente trois,https://www.booking.com/hotel/fr/le-trente-tro...,3 sur 5,"Situé à Uzès, l'établissement Le trente trois ...",44.009642,4.417885
697,Uzes,La Taverne,https://www.booking.com/hotel/fr/la-taverne.fr...,2 sur 5,"La Taverne est située à Uzès, dans une rue pié...",44.014659,4.420051
698,Uzes,Le Cherche Midi - Appartement centre historiqu...,https://www.booking.com/hotel/fr/appartement-c...,3 sur 5,Hébergement géré par un particulier,44.012710,4.418500
